In [1]:
import gensim
from gensim.models.callbacks import CallbackAny2Vec
import csv
import os
import time


# data files
playlists_file =  'csvs/playlists.csv'
tracks_file =  'csvs/tracks_wop.csv'

csv.field_size_limit(1000000)

tracks = {}
temp = True
test = None
with open(tracks_file, 'r', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    for row in spamreader:
        col = str(row)[2:-2].split(';')
        tracks[col[0]] = [col[1] + ' - ' + col[2]]
        if temp:
            test = col[0]
            temp = False
        # track ID = title - artist

print(f'Number of tracks: {len(tracks)}')
print(f'ID {test} - {tracks[test]}')

playlists = []
with open(playlists_file, 'r', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    for row in spamreader:
        columns = str(row)[2:-2].split(';')
        playlist = []
        for col in columns[2:]:
            playlist.append(col)
        playlists.append(playlist)

print(f'Number of playlists: {len(playlists)}')
print('playlists[0] track IDs')
for track in playlists[0][:5]:
    print(f'ID {track} - {tracks[track]}')
print('...')

Number of tracks: 2562609
ID 2NDMLu8ZNrAsAsPAoW5VOx - ['Raveena - Tweety']
Number of playlists: 50800
playlists[0] track IDs
ID 2NDMLu8ZNrAsAsPAoW5VOx - ['Raveena - Tweety']
ID 43tQOLiTfoVwURdx5Ei7yr - ['Deb Never - Someone Else']
ID 1B6Za6s7tPvxU9ISUEgRda - ['JVNA - Breakdown']
ID 4fRfpZ6lNrraY9ZnXWt8rb - ['Mike Shinoda - License to Waltz']
ID 4LrE3GPel06vD9stHrRhcg - ['Shawn Wasabi - the snack that smiles back (feat. raychel jay)']
...


In [2]:
# hyperparameters

min_count = 2      # minimum number of occurences of a track in all playlists
window = 2          # length of sequences
embedding_dim = 100 # number of dimensions in hidden layer
batch_words = 10000 # number of tracks to process in each batch
iter = 15           # number of iterations
sg = 0              # skip-gram (1) or Continuous Bag Of Words (0)

# ------------------------------------------------------------------------------

valid_examples = [
    '2NMgVh5qaPprKTEzFe3501', # The Police - Roxanne
    '3Ti0GdlrotgwsAVBBugv0I', # A Tribe Called Quest - Can I Kick It?
    '0nyrltZrQGAJMBZc1bYvuQ', # James Brown - Get Up Offa That Thing
    '4hy4fb5D1KL50b3sng9cjw', # Nirvana - Smells Like Teen Spirit
    '1P49MJhU5vzttesFxw3dOM', # Bob Marley & The Wailers - Three Little Birds
    '76GlO5H5RT6g7y0gev86Nk', # The Cure - Just Like Heaven
    '40tAOP3DPqmVD6L1h45Jp6', # Frank Sinatra - My Way
    '4IMvgp0WZqr9mRqpEvDKxI', # The Clash - Rock the Casbah
    '1iDcKYNvo6gglrOG6lvnHL', # The Rolling Stones - Sympathy For The Devil
    '5uvosCdMlFdTXhoazkTI5R', # The Doors - Light My Fire
    '15JINEqzVMv3SvJTAXAKED', # Eminem - Love The Way You Lie
    '69kOkLUCkxIZYexIgSG8rq', # Daft Punk - Get Lucky
    '6oVY50pmdXqLNVeK8bzomn', # John Coltrane - My Favorite Things
    '6ui6l3ZNvlrGQZArwo8195', # Sex Pistols - God Save The Queen
    '0YammaEkYSeo9vQYZ1OwS6', # David Guetta - Say My Name
    '4SHZsQIdS2N1E5yqvoXF8o'  # Andy Williams - Can't Take My Eyes Off You
]

# ------------------------------------------------------------------------------

class logger(CallbackAny2Vec):
    def __init__(self):
        print('Starting...')
        self.epoch = 0
        self.loss = 0

    def on_train_begin(self, model):
        self.start = time.time()

    def on_epoch_end(self, model):
        elapsed = time.time() - self.start
        print('#{}'.format(self.epoch), 'loss =',
              (model.get_latest_training_loss() - self.loss) / batch_words,
              'elapsed time =', elapsed // 60, 'minutes', elapsed % 60, 'seconds')
        self.epoch += 1
        self.loss = model.get_latest_training_loss()
        print('Saving model...')
        model.save('word2vec.model')
        _model = gensim.models.Word2Vec.load('word2vec.model')
        for track in valid_examples:
            similar = _model.wv.most_similar(positive=[track], topn=8)
            most_similar = ''
            for i in range(0, 8):
                most_similar = most_similar + '%s (%.2f)' % (tracks[similar[i][0]][0], similar[i][1]) + ', '
            print('  %s -> %s' % (tracks[track][0], most_similar))
        print()
        del _model

In [3]:
model = gensim.models.Word2Vec(sentences=playlists, size=embedding_dim,
                               min_count=min_count, window=window, iter=iter,
                               batch_words=batch_words,compute_loss=True,
                               callbacks=[logger()])

Starting...
#0 loss = 723.4469 elapsed time = 0.0 minutes 10.930516719818115 seconds
Saving model...
  The Police - Roxanne -> Gökşin Derin - Benimle Kalsan? (0.43), George Salazar - Michael in the Bathroom (0.43), NERVO - Worlds Collide - Edison Cole Remix (0.43), Wilkinson - Breathe (0.42), Isla Vista Worship - Opened Up the Heavens (0.42), UVERworld - Colors of the Heart (0.42), Sơn Tùng M-TP - Chúng Ta Không Thuộc Về Nhau (0.41), MC Cego Abusado - Cafajeste (0.41), 
  A Tribe Called Quest - Can I Kick It? -> Oscar Williams and the Band of Life - Heal Our Land (0.46), Skinny Puma - 19 (0.45), Joanna Wang - You've Got a Friend - Acoustic version (0.44), Ricardo Montaner - Me va a extrañar (aka Unchain My Heart) (0.43), Chico Che y La Crisis - Quien Pompo (0.43), Ikke Hüftgold - Urensohn (0.43), Ooyy - Mt. Fuji (0.42), Idealism - illusions (0.42), 
  James Brown - Get Up Offa That Thing -> Björk - It's Oh So Quiet (0.46), Alessandra Amoroso - La stessa (0.43), Edith Márquez - Procuro 

  Eminem - Love The Way You Lie -> Macklemore & Ryan Lewis - Can't Hold Us - feat. Ray Dalton (0.97), Flo Rida - My House (0.97), Plain White T's - Hey There Delilah (0.97), Iyaz - Replay (0.97), Avicii - Wake Me Up (0.97), Calvin Harris - My Way (0.96), Far East Movement - Rocketeer (0.96), Beyoncé - Love On Top (0.96), 
  Daft Punk - Get Lucky (feat. Pharrell Williams & Nile Rodgers) -> Mark Ronson - Uptown Funk (feat. Bruno Mars) (0.94), Avicii - Wake Me Up (0.93), Macklemore & Ryan Lewis - Can't Hold Us - feat. Ray Dalton (0.93), The Wanted - Glad You Came (0.93), Shakira - Waka Waka (This Time for Africa) [The Official 2010 FIFA World Cup (TM) Song] (feat. Freshlyground) (0.93), Taio Cruz - Dynamite (0.93), Miley Cyrus - We Can't Stop (0.92), Far East Movement - Like A G6 (0.92), 
  John Coltrane - My Favorite Things -> Mrs. GREEN APPLE - lovin' (0.46), Gzuz - Drück Drück (0.46), Trinidad Cardona - Show Off (0.43), Red Hot Chili Peppers - Higher Ground (0.43), Anirudh Ravichander 

#3 loss = 8.2688 elapsed time = 0.0 minutes 57.074913024902344 seconds
Saving model...
  The Police - Roxanne -> Depeche Mode - Enjoy the Silence (0.98), The Runaways - Cherry Bomb (0.98), David Bowie - Let's Dance - 1999 Remaster (0.98), Chuck Berry - Johnny B. Goode (0.98), The Buggles - Video Killed The Radio Star (0.98), The Wallflowers - One Headlight (0.97), The Beatles - Drive My Car - Remastered 2009 (0.97), Christopher Cross - Arthur's Theme (Best That You Can Do) (0.97), 
  A Tribe Called Quest - Can I Kick It? -> NEFFEX - Fight Back (0.99), dvsn - With Me (0.99), Chaka Khan - I Feel for You (0.99), Burna Boy - On the Low (0.99), Jon Lajoie - Everyday Normal Guy 2 (0.99), Chris Brown - Bitches N Marijuana (feat. ScHoolboy Q) (0.99), 2Pac - Dear Mama (0.99), Skrillex - Would You Ever (0.99), 
  James Brown - Get Up Offa That Thing -> Fugees - Killing Me Softly With His Song (0.95), Jax Jones - Instruction (feat. Demi Lovato & Stefflon Don) (0.95), Collective Soul - Shine (0.95

  Frank Sinatra - My Way -> George Benson - Give Me the Night (1.00), Bobby Darin - Beyond the Sea (1.00), White Town - Your Woman (1.00), Jamiroquai - Little L (1.00), Deichkind - Bück dich hoch (1.00), Etta James - I'd Rather Go Blind (1.00), Echo & the Bunnymen - The Killing Moon (1.00), The Jam - Town Called Malice (1.00), 
  The Clash - Rock the Casbah -> Bonnie Tyler - Total Eclipse of the Heart (0.98), Lionel Richie - Say You, Say Me (0.98), Air Supply - All Out of Love (0.98), Eric Clapton - Tears in Heaven - Acoustic Live at MTV Unplugged, Bray Film Studios, Windsor, England, UK, 1/16/1992 2013 Remaster (0.98), Bruce Springsteen - Streets of Philadelphia (0.98), Elvis Presley - Hound Dog (0.98), Elvis Presley - Jailhouse Rock (0.98), Prince - Raspberry Beret (0.98), 
  The Rolling Stones - Sympathy For The Devil -> Elton John - Rocket Man (I Think It's Going To Be A Long, Long Time) (0.99), Electric Light Orchestra - Mr. Blue Sky (0.99), The Rolling Stones - Gimme Shelter (0.9

  Daft Punk - Get Lucky (feat. Pharrell Williams & Nile Rodgers) -> Daft Punk - Get Lucky (feat. Pharrell Williams & Nile Rodgers) - Radio Edit (1.00), Robin Thicke - Blurred Lines (1.00), Major Lazer - Lean On (feat. MØ & DJ Snake) (1.00), Years & Years - King (1.00), Olly Murs - Troublemaker (feat. Flo Rida) (1.00), Enrique Iglesias - I Like It (1.00), Beyoncé - Single Ladies (Put a Ring on It) (1.00), Example - Kickstarts (1.00), 
  John Coltrane - My Favorite Things -> Robin Hustin - Light It Up (1.00), ILLENIUM - Beautiful Creatures (feat. MAX) (1.00), Vicetone - Nothing Stopping Me (feat. Kat Nestel) (1.00), Hardwell - Shine A Light (1.00), Thomas Rhett - Look What God Gave Her (1.00), Diviners - Savannah (1.00), Leo Santana - Crush Blogueirinha (1.00), 5 Seconds of Summer - Easier - Seeb Remix (1.00), 
  Sex Pistols - God Save the Queen -> Van Halen - Unchained - 2015 Remaster (1.00), Sam The Sham & The Pharaohs - Wooly Bully (1.00), Filter - Hey Man, Nice Shot - Remastered Vers

...

  Eminem - Love The Way You Lie -> Bad Meets Evil - Lighters (0.99), Lil Wayne - Mirror (0.99), Lil Wayne - How To Love (0.99), Lil Wayne - How To Love (0.99), Wildstylez - Year Of Summer (0.99), AJR - I'm Ready (0.99), Macklemore - Downtown (feat. Melle Mel, Grandmaster Caz, Kool Moe Dee & Eric Nally) (0.99), Rihanna - FourFiveSeconds (0.99), 
  Daft Punk - Get Lucky (feat. Pharrell Williams & Nile Rodgers) -> Daft Punk - Get Lucky (feat. Pharrell Williams & Nile Rodgers) - Radio Edit (1.00), Ylvis - The Fox (What Does the Fox Say?) (0.99), Samantha Jade - What You've Done to Me (0.99), Bob Sinclar - Love Generation - Radio Edit (0.99), Major Lazer - Lean On (feat. MØ & DJ Snake) (0.99), CeeLo Green - Fuck You (0.99), Robin Thicke - Blurred Lines (0.99), Avicii - Wake Me Up (0.99), 
  John Coltrane - My Favorite Things -> Chiclete Com Banana - Diga Que Valeu (1.00), SFDK - Ternera Podrida (Instrumental) (1.00), Dusty Kid - Sysma - Original Mix (1.00), Lane 8 - Feld (1.00), Marcela Man

#10 loss = 187.6338 elapsed time = 2.0 minutes 38.3990843296051 seconds
Saving model...
  The Police - Roxanne -> Don McLean - American Pie (1.00), Tom Petty - Free Fallin' (1.00), Bruce Springsteen - Streets of Philadelphia - Single Edit (1.00), Sting - Shape Of My Heart (1.00), Paul Simon - Graceland (1.00), Journey - Faithfully (1.00), The Beatles - Come Together - Remastered (1.00), Billy Idol - Mony Mony (1.00), 
  A Tribe Called Quest - Can I Kick It? -> Skee-Lo - I Wish (0.99), LL Cool J - Mama Said Knock You Out (0.99), Fugees - Ready or Not (0.99), Eminem - My Name Is (0.99), Outkast - Roses (0.99), Dr. Dre - Bang Bang (0.98), JAY-Z - Hard Knock Life (Ghetto Anthem) (0.98), Skee-Lo - I Wish - Radio Edit (0.98), 
  James Brown - Get Up Offa That Thing -> Londonbeat - I've Been Thinking About You (1.00), Marvin Gaye - Sexual Healing (1.00), Bob Marley & The Wailers - No Woman, No Cry - Live At The Lyceum, London/1975 (1.00), Sting - Fragile (1.00), Chumbawamba - Tubthumping (1.0

  Daft Punk - Get Lucky (feat. Pharrell Williams & Nile Rodgers) -> Daft Punk - Get Lucky (feat. Pharrell Williams & Nile Rodgers) - Radio Edit (1.00), Ylvis - The Fox (What Does the Fox Say?) (0.99), Loreen - Euphoria (0.99), Shawn Mendes - Air (0.99), Sunrise Avenue - Hollywood Hills (0.99), Pitbull - Timber (feat. Ke$ha) (0.99), Toploader - Dancing in the Moonlight (0.98), Rameez - Girls in Luv (0.98), 
  John Coltrane - My Favorite Things -> Klamydia - Pienen pojan elämää (1.00), Alan Jackson - The Older I Get (1.00), Above & Beyond - Sahara Love - Seven Lions Remix (1.00), Edamame - Tongue (1.00), World Mestizo Ensemble presents Acoustic - Self Control (1.00), Hans Zimmer - Like a Dog Chasing Cars (1.00), Jizzel Noel - Coupe Talk (1.00), Banda Pequeños Musical - Regálame (1.00), 
  Sex Pistols - God Save the Queen -> Loverboy - Turn Me Loose (1.00), KISS - Shout It Out Loud (1.00), Rainbow - The Temple Of The King (1.00), Cheap Trick - Surrender (1.00), Harry Chapin - Cats in the 

  The Cure - Just like Heaven -> The Cure - Lovesong - 2010 Remaster (0.99), Fleetwood Mac - Gypsy (0.99), The Smiths - Bigmouth Strikes Again - 2011 Remaster (0.99), Nancy Sinatra - Bang Bang (My Baby Shot Me Down) (0.99), The Cure - Boys Don't Cry - Single Version (0.99), The Beatles - Michelle - Remastered 2009 (0.99), The Animals - The House of the Rising Sun (0.99), Queen - Somebody To Love (0.99), 
  Frank Sinatra - My Way -> The Isley Brothers - This Old Heart Of Mine (Is Weak For You) (1.00), Madonna - Crazy for You - Edit (1.00), Roxette - Spending My Time (1.00), Barry White - Just The Way You Are (1.00), Elton John - Don't Let The Sun Go Down On Me (1.00), The Alan Parsons Project - Don't Answer Me (1.00), Frank Sinatra - New York, New York (1.00), Otis Redding - (Sittin' On) the Dock of the Bay (1.00), 
  The Clash - Rock the Casbah -> Fine Young Cannibals - She Drives Me Crazy (1.00), Daryl Hall & John Oates - Maneater (1.00), Depeche Mode - Enjoy the Silence (1.00), Madne

  Daft Punk - Get Lucky (feat. Pharrell Williams & Nile Rodgers) -> Daft Punk - Get Lucky (feat. Pharrell Williams & Nile Rodgers) - Radio Edit (1.00), Ylvis - The Fox (What Does the Fox Say?) (0.99), Shawn Mendes - Air (0.98), Loreen - Euphoria (0.98), Daft Punk - Lose Yourself to Dance (feat. Pharrell Williams) (0.98), Sunrise Avenue - Hollywood Hills (0.98), Pitbull - Timber (feat. Ke$ha) (0.98), Olly Murs - Wrapped Up (feat. Travie McCoy) (0.98), 
  John Coltrane - My Favorite Things -> Alan Jackson - The Older I Get (1.00), Edamame - Tongue (1.00), Lazerhawk - Overdrive (1.00), Klamydia - Pienen pojan elämää (1.00), World Mestizo Ensemble presents Acoustic - Self Control (1.00), Emil Rottmayer - Mega (Bonus Track) (1.00), Weval - Gimme Some (1.00), El Reino Infantil - Bartolito (1.00), 
  Sex Pistols - God Save the Queen -> Loverboy - Turn Me Loose (1.00), KISS - Shout It Out Loud (1.00), Cheap Trick - Surrender (1.00), Donovan - Sunshine Superman (1.00), Harry Chapin - Cats in th

In [4]:
def search_track(search=''):
    for track in tracks:
        if tracks[track][0].lower().find(search.lower()) != -1 and track in model.wv.vocab:
            print(f'{track} : {tracks[track][0]}') # ' : {tracks[track][1]}')

In [5]:


# hyperparameters

min_count = 10      # minimum number of occurences of a track in all playlists
window = 2          # length of sequences
embedding_dim = 100 # number of dimensions in hidden layer
batch_words = 10000 # number of tracks to process in each batch
iter = 15           # number of iterations
sg = 0              # skip-gram (1) or Continuous Bag Of Words (0)

# ------------------------------------------------------------------------------

valid_examples = [
    '2NMgVh5qaPprKTEzFe3501', # The Police - Roxanne
    '3Ti0GdlrotgwsAVBBugv0I', # A Tribe Called Quest - Can I Kick It?
    '0nyrltZrQGAJMBZc1bYvuQ', # James Brown - Get Up Offa That Thing
    '4hy4fb5D1KL50b3sng9cjw', # Nirvana - Smells Like Teen Spirit
    '1P49MJhU5vzttesFxw3dOM', # Bob Marley & The Wailers - Three Little Birds
    '76GlO5H5RT6g7y0gev86Nk', # The Cure - Just Like Heaven
    '40tAOP3DPqmVD6L1h45Jp6', # Frank Sinatra - My Way
    '4IMvgp0WZqr9mRqpEvDKxI', # The Clash - Rock the Casbah
    '1iDcKYNvo6gglrOG6lvnHL', # The Rolling Stones - Sympathy For The Devil
    '5uvosCdMlFdTXhoazkTI5R', # The Doors - Light My Fire
    '15JINEqzVMv3SvJTAXAKED', # Eminem - Love The Way You Lie
    '69kOkLUCkxIZYexIgSG8rq', # Daft Punk - Get Lucky
    '6oVY50pmdXqLNVeK8bzomn', # John Coltrane - My Favorite Things
    '6ui6l3ZNvlrGQZArwo8195', # Sex Pistols - God Save The Queen
    '0YammaEkYSeo9vQYZ1OwS6', # David Guetta - Say My Name
    '4SHZsQIdS2N1E5yqvoXF8o'  # Andy Williams - Can't Take My Eyes Off You
]

# ------------------------------------------------------------------------------

class logger(CallbackAny2Vec):
    def __init__(self):
        print('Starting...')
        self.epoch = 0
        self.loss = 0

    def on_train_begin(self, model):
        self.start = time.time()

    def on_epoch_end(self, model):
        elapsed = time.time() - self.start
        print('#{}'.format(self.epoch), 'loss =',
              (model.get_latest_training_loss() - self.loss) / batch_words,
              'elapsed time =', elapsed // 60, 'minutes', elapsed % 60, 'seconds')
        self.epoch += 1
        self.loss = model.get_latest_training_loss()
        print('Saving model...')
        model.save('word2vec.model')
        _model = gensim.models.Word2Vec.load('word2vec.model')
        for track in valid_examples:
            similar = _model.wv.most_similar(positive=[track], topn=8)
            most_similar = ''
            for i in range(0, 8):
                most_similar = most_similar + '%s (%.2f)' % (tracks[similar[i][0]][0], similar[i][1]) + ', '
            print('  %s -> %s' % (tracks[track][0], most_similar))
        print()
        del _model

# invert track dictionary so we can use ID or name
track_to_id = {}
for track in tracks:
    track_to_id[tracks[track][0]] = track

i = 0
for track in track_to_id:
    print(f'{track} : ID {track_to_id[track]}')
    if i > 5:
        break
    i += 1

Raveena - Tweety : ID 2NDMLu8ZNrAsAsPAoW5VOx
Deb Never - Someone Else : ID 43tQOLiTfoVwURdx5Ei7yr
JVNA - Breakdown : ID 1B6Za6s7tPvxU9ISUEgRda
Mike Shinoda - License to Waltz : ID 43c3rKUgqrOx9GKCbeCSUJ
Shawn Wasabi - the snack that smiles back (feat. raychel jay) : ID 6gfDkakl2eWMuGSiOyStC3
anders - Bad Habits : ID 2o4zCAvc73Gr4wdOSN4Mei
Dexter King - Heartless : ID 09aRWpXE2dS3Fb27znVoNy


In [6]:
seed_tracks = [
    '5wQnmLuC1W7ATsArWACrgW',
    '3Zwu2K0Qa5sT6teCCHPShP',
    '4HcDu9IGRCvSTJE4yW5Ugv',
    '0d28khcov6AiegSCpG5TuT',
]

num_tracks = 100
seed_window = num_tracks
for i in range(0, num_tracks - len(seed_tracks)):
    next_track = model.wv.most_similar(positive=seed_tracks[-seed_window:], topn=1)
    if next_track == None:
        break
    seed_tracks.append(next_track[0][0])
for track in seed_tracks:
    print(f'{track} : {tracks[track][0]}')

5wQnmLuC1W7ATsArWACrgW : My Chemical Romance - Welcome to the Black Parade
3Zwu2K0Qa5sT6teCCHPShP : Fall Out Boy - Thnks fr th Mmrs
4HcDu9IGRCvSTJE4yW5Ugv : Linkin Park - Breaking the Habit
0d28khcov6AiegSCpG5TuT : Gorillaz - Feel Good Inc.
1AdYZ6X00nXmO613Y7GJOl : blink-182 - I Miss You
1fJFuvU2ldmeAm5nFIHcPP : blink-182 - First Date
1oTo3ijRbaDAtrjJrGAPSw : blink-182 - I Miss You
0J69x3mqm7U6tBPKsjpsWR : All Time Low - Dear Maria, Count Me In
47No93LxERvV6MtOAmQzHS : Paramore - Ignorance
0a7BloCiNzLDD9qSQHh5m7 : Fall Out Boy - Dance, Dance
1HNE2PX70ztbEl6MLxrpNL : Sum 41 - In Too Deep
7HGTkn2aX7MNdKs7nV2xBt : The Killers - Somebody Told Me
0MsrWnxQZxPAcov7c74sSo : Green Day - Holiday / Boulevard of Broken Dreams
3YsGKsKkWwAAdjPeT9GeiF : blink-182 - I Miss You
23oxJmDc1V9uLUSmN2LIvx : Yellowcard - Ocean Avenue
6vFD3c1WI1zuKoyk22dMw3 : Good Charlotte - Little Things
3ZsexY07D4t4HRq7ogeSAS : All Time Low - Dear Maria, Count Me In
28IEbk5a7twNTbUEvWslUb : Finger Eleven - Paralyzer
1yKu2M